# Adam
:label:`sec_adam`

In the discussions leading up to this section we encountered a number of techniques for efficient optimization. Let us recap them in detail here:

* We saw that :numref:`sec_sgd` is more effective than Gradient Descent when solving optimization problems, e.g., due to its inherent resilience to redundant data. 
* We saw that :numref:`sec_minibatch_sgd` affords significant additional efficiency arising from vectorization, using larger sets of observations in one minibatch. This is the key to efficient multi-machine, multi-GPU and overall parallel processing. 
* :numref:`sec_momentum` added a mechanism for aggregating a history of past gradients to accelerate convergence.
* :numref:`sec_adagrad` used per-coordinate scaling to allow for a computationally efficient preconditioner. 
* :numref:`sec_rmsprop` decoupled per-coordinate scaling from a learning rate adjustment. 

Adam :cite:`Kingma.Ba.2014` combines all these techniques into one efficient learning algorithm. As expected, this is an algorithm that has become rather popular as one of the more robust and effective optimization algorithms to use in deep learning. It is not without issues, though. In particular, :cite:`Reddi.Kale.Kumar.2019` show that there are situations where Adam can diverge due to poor variance control. In a follow-up work :cite:`Zaheer.Reddi.Sachan.ea.2018` proposed a hotfix to Adam, called Yogi which addresses these issues. More on this later. For now let us review the Adam algorithm. 

## The Algorithm

One of the key components of Adam is that it uses exponential weighted moving averages (also known as leaky averaging) to obtain an estimate of both the momentum and also the second moment of the gradient. That is, it uses the state variables

$$\begin{aligned}
    \mathbf{v}_t & \leftarrow \beta_1 \mathbf{v}_{t-1} + (1 - \beta_1) \mathbf{g}_t, \\
    \mathbf{s}_t & \leftarrow \beta_2 \mathbf{s}_{t-1} + (1 - \beta_2) \mathbf{g}_t^2.
\end{aligned}$$

Here $\beta_1$ and $\beta_2$ are nonnegative weighting parameters. Common choices for them are $\beta_1 = 0.9$ and $\beta_2 = 0.999$. That is, the variance estimate moves *much more slowly* than the momentum term. Note that if we initialize $\mathbf{v}_0 = \mathbf{s}_0 = 0$ we have a significant amount of bias initially towards smaller values. This can be addressed by using the fact that $\sum_{i=0}^t \beta^i = \frac{1 - \beta^t}{1 - \beta}$ to re-normalize terms. Correspondingly the normalized state variables are given by 

$$\hat{\mathbf{v}}_t = \frac{\mathbf{v}_t}{1 - \beta_1^t} \text{ and } \hat{\mathbf{s}}_t = \frac{\mathbf{s}_t}{1 - \beta_2^t}.$$

Armed with the proper estimates we can now write out the update equations. First, we rescale the gradient in a manner very much akin to that of RMSProp to obtain

$$\mathbf{g}_t' = \frac{\eta \hat{\mathbf{v}}_t}{\sqrt{\hat{\mathbf{s}}_t} + \epsilon}.$$

Unlike RMSProp our update uses the momentum $\hat{\mathbf{v}}_t$ rather than the gradient itself. Moreover, there is a slight cosmetic difference as the rescaling happens using $\frac{1}{\sqrt{\hat{\mathbf{s}}_t} + \epsilon}$ instead of $\frac{1}{\sqrt{\hat{\mathbf{s}}_t + \epsilon}}$. The former works arguably slightly better in practice, hence the deviation from RMSProp. Typically we pick $\epsilon = 10^{-6}$ for a good trade-off between numerical stability and fidelity. 

Now we have all the pieces in place to compute updates. This is slightly anticlimactic and we have a simple update of the form

$$\mathbf{x}_t \leftarrow \mathbf{x}_{t-1} - \mathbf{g}_t'.$$

Reviewing the design of Adam its inspiration is clear. Momentum and scale are clearly visible in the state variables. Their rather peculiar definition forces us to debias terms (this could be fixed by a slightly different initialization and update condition). Second, the combination of both terms is pretty straightforward, given RMSProp. Last, the explicit learning rate $\eta$ allows us to control the step length to address issues of convergence. 

## Implementation 

Implementing Adam from scratch is not very daunting. For convenience we store the timestep counter $t$ in the `hyperparams` dictionary. Beyond that all is straightforward.


In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.GradDescUtils.plotGammas
import jp.live.ugai.d2j.util.GradDescUtils.train2d
import jp.live.ugai.d2j.util.GradDescUtils.showTrace2d
import jp.live.ugai.d2j.util.TrainingChapter11.getDataCh11
import jp.live.ugai.d2j.util.TrainingChapter11.plotLossEpoch
import jp.live.ugai.d2j.util.TrainingChapter11.trainCh11
import jp.live.ugai.d2j.util.TrainingChapter11.trainConciseCh11
import jp.live.ugai.d2j.util.LossTime

In [2]:
fun initAdamStates(featureDimension: Int): NDList {
    val manager = NDManager.newBaseManager();
    val vW = manager.zeros(Shape(featureDimension.toLong(), 1));
    val vB = manager.zeros(Shape(1));
    val sW = manager.zeros(Shape(featureDimension.toLong(), 1));
    val sB = manager.zeros(Shape(1));
    return NDList(vW, sW, vB, sB)
}

object Optimization {
    fun adam(params:NDList , states:NDList , hyperparams:MutableMap<String, Float> ) {
        val beta1 = 0.9f;
        val beta2 = 0.999f;
        val eps = 1e-6.toFloat()
        val time = hyperparams.get("time")!!
        for (i in 0 until params.size) {
            val param = params.get(i);
            val velocity = states.get(2 * i);
            val state = states.get(2 * i + 1);
            // Update parameter, velocity, and state
            // velocity = beta1 * v + (1 - beta1) * param.gradient
            velocity.muli(beta1).addi(param.getGradient().mul(1 - beta1));
            // state = beta2 * state + (1 - beta2) * param.gradient^2
            state.muli(beta2).addi(param.getGradient().square().mul(1 - beta2));
            // vBiasCorr = velocity / ((1 - beta1)^(time))
            val vBiasCorr = velocity.div(1 - Math.pow(beta1.toDouble(), time.toDouble()));
            // sBiasCorr = state / ((1 - beta2)^(time))
            val sBiasCorr = state.div(1 - Math.pow(beta2.toDouble(), time.toDouble()))
            // param -= lr * vBiasCorr / (sBiasCorr^(1/2) + eps)
            param.subi(vBiasCorr.mul(hyperparams.get("lr")).div(sBiasCorr.sqrt().add(eps)));
        }
        hyperparams.put("time", time + 1);
    }
}

We are ready to use Adam to train the model. We use a learning rate of $\eta = 0.01$.


In [7]:
val airfoil = getDataCh11(10, 1500);

fun trainAdam(lr: Float, time: Float, numEpochs: Int) : LossTime {
    val featureDimension = airfoil.getColumnNames().size
    val hyperparams = mutableMapOf<String, Float>()
    hyperparams.put("lr", lr);
    hyperparams.put("time", time);
    return trainCh11(Optimization::adam, 
                                       initAdamStates(featureDimension), 
                                       hyperparams, airfoil, 
                                       featureDimension, numEpochs);
}

val lossTime = trainAdam(0.01f, 1.0f, 2);
plotLossEpoch(lossTime.loss, lossTime.epoch)

loss: 0.248, 0.132 sec/epoch


A more concise implementation is straightforward since `adam` is one of the algorithms provided as part of the DJL optimization library.

We will set the learning rate to 0.01f to remain consistent with the previous section.
However, you typically won't need to set this yourself as the default will usually work fine.

In [8]:
val lrt = Tracker.fixed(0.01f);
val adam = Optimizer.adam().optLearningRateTracker(lrt).build();

val lossTime = trainConciseCh11(adam, airfoil, 2);
plotLossEpoch(lossTime.loss, lossTime.epoch)

Training:    100% |████████████████████████████████████████| Accuracy: 0.68, L2Loss: 0.29��███████████████████               | Accuracy: 0.67, L2Loss: 0.33███         | Accuracy: 0.67, L2Loss: 0.31███████████████████████████████   | Accuracy: 0.68, L2Loss: 0.30
Training:    100% |████████████████████████████████████████| Accuracy: 0.67, L2Loss: 0.27�█                  | Accuracy: 0.67, L2Loss: 0.28g:     96% |███████████████████████████████████████ | Accuracy: 0.67, L2Loss: 0.27
loss: 0.243, 0.202 sec/epoch


## Yogi

One of the problems of Adam is that it can fail to converge even in convex settings when the second moment estimate in $\mathbf{s}_t$ blows up. As a fix :cite:`Zaheer.Reddi.Sachan.ea.2018` proposed a refined update (and initialization) for $\mathbf{s}_t$. To understand what's going on, let us rewrite the Adam update as follows:

$$\mathbf{s}_t \leftarrow \mathbf{s}_{t-1} + (1 - \beta_2) \left(\mathbf{g}_t^2 - \mathbf{s}_{t-1}\right).$$

Whenever $\mathbf{g}_t^2$ has high variance or updates are sparse, $\mathbf{s}_t$ might forget past values too quickly. A possible fix for this is to replace $\mathbf{g}_t^2 - \mathbf{s}_{t-1}$ by $\mathbf{g}_t^2 \odot \mathop{\mathrm{sgn}}(\mathbf{g}_t^2 - \mathbf{s}_{t-1})$. Now the magnitude of the update no longer depends on the amount of deviation. This yields the Yogi updates

$$\mathbf{s}_t \leftarrow \mathbf{s}_{t-1} + (1 - \beta_2) \mathbf{g}_t^2 \odot \mathop{\mathrm{sgn}}(\mathbf{g}_t^2 - \mathbf{s}_{t-1}).$$

The authors furthermore advise to initialize the momentum on a larger initial batch rather than just initial pointwise estimate. We omit the details since they are not material to the discussion and since even without this convergence remains pretty good.


In [10]:
object Optimization {
    fun yogi(params: NDList , states: NDList , hyperparams: MutableMap<String, Float> ) {
        val beta1 = 0.9f;
        val beta2 = 0.999f;
        val eps = 1e-3.toFloat()
        val time = hyperparams.get("time")!!
        for (i in 0 until params.size) {
            val param = params.get(i);
            val velocity = states.get(2 * i);
            val state = states.get(2 * i + 1);
            // Update parameter, velocity, and state
            // velocity = beta1 * v + (1 - beta1) * param.gradient
            velocity.muli(beta1).addi(param.getGradient().mul(1 - beta1));
            /* Rewritten Update */
            // state = state + (1 - beta2) * sign(param.gradient^2 - state) 
            //         * param.gradient^2
            state.addi(param.getGradient().square().sub(state).sign().mul(1 - beta2));
            // vBiasCorr = velocity / ((1 - beta1)^(time))
            val vBiasCorr = velocity.div(1 - Math.pow(beta1.toDouble(), time.toDouble()));
            // sBiasCorr = state / ((1 - beta2)^(time))
            val sBiasCorr = state.div(1 - Math.pow(beta2.toDouble(), time.toDouble()));
            // param -= lr * vBiasCorr / (sBiasCorr^(1/2) + eps)
            param.subi(vBiasCorr.mul(hyperparams.get("lr")).div(sBiasCorr.sqrt().add(eps)));
        }
        hyperparams.put("time", time + 1);
    }
}

In [12]:
val airfoil = getDataCh11(10, 1500);

fun trainYogi(lr: Float, time: Float, numEpochs: Int) : LossTime {
    val featureDimension = airfoil.getColumnNames().size
    val hyperparams = mutableMapOf<String, Float>();
    hyperparams.put("lr", lr)
    hyperparams.put("time", time)
    return trainCh11(Optimization::yogi, 
                                       initAdamStates(featureDimension), 
                                       hyperparams, airfoil, 
                                       featureDimension, numEpochs);
}

val lossTime = trainYogi(0.01f, 1.0f, 2)
plotLossEpoch(lossTime.loss, lossTime.epoch)

loss: 0.245, 0.144 sec/epoch


## Summary

* Adam combines features of many optimization algorithms into a fairly robust update rule. 
* Created on the basis of RMSProp, Adam also uses EWMA on the minibatch stochastic gradient
* Adam uses bias correction to adjust for a slow startup when estimating momentum and a second moment. 
* For gradients with significant variance we may encounter issues with convergence. They can be amended by using larger minibatches or by switching to an improved estimate for $\mathbf{s}_t$. Yogi offers such an alternative. 

## Exercises

1. Adjust the learning rate and observe and analyze the experimental results.
1. Can you rewrite momentum and second moment updates such that it does not require bias correction?
1. Why do you need to reduce the learning rate $\eta$ as we converge?
1. Try to construct a case for which Adam diverges and Yogi converges?
